# N-Net医療CT画像処理 - トレーニングノートブック

## 概要
このノートブックは4D-CBCT画像の伪影除去を目的としたN-Netモデルのトレーニングを実行します。

## 実行手順
1. **環境準備**: ライブラリのインポートとプロジェクト設定
2. **実験設定**: 再現性確保と実験ディレクトリ作成
3. **ハードウェア設定**: GPU/CPU使用設定とメトリクス初期化
4. **ロギング設定**: TensorBoard・W&B設定
5. **データ準備**: データセット作成とローダー設定
6. **モデル構築**: N-Netアーキテクチャの初期化
7. **最適化設定**: オプティマイザー・損失関数設定
8. **トレーニング実行**: メインループとバリデーション
9. **結果保存**: ベストモデル保存と可視化
10. **リソース解放**: メモリとセッションクリーンアップ

## 注意事項
- GPU メモリ不足時は batch_size を調整してください
- AMP (Automatic Mixed Precision) を使用してメモリ効率を改善
- キャッシュ率は使用可能メモリに応じて調整可能

## 1. 環境準備・ライブラリインポート

**目的**: プロジェクトルートの設定と必要なライブラリのインポート
- プロジェクトルートをPythonパスに追加
- 医療画像処理・深層学習関連ライブラリをインポート
- 警告メッセージの抑制設定

In [ ]:
import sys
import os

# プロジェクトルートの設定と確認
notebook_dir = os.path.dirname(os.path.abspath(""))
project_root = os.path.abspath(os.path.join(notebook_dir, "../.."))

# Pythonパスに追加（重複チェック）
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"プロジェクトルートをPythonパスに追加: {project_root}")
else:
    print(f"プロジェクトルートは既にPythonパスに存在: {project_root}")

# ディレクトリ存在確認
if not os.path.exists(project_root):
    raise FileNotFoundError(f"プロジェクトルートが見つかりません: {project_root}")

project_root

In [5]:
import shutil
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import MSELoss, L1Loss
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary
import torchvision
import warnings
from datetime import datetime
import wandb

# MONAI関連ライブラリ
from monai.losses import SSIMLoss, PerceptualLoss
from monai.utils import set_determinism
from monai.transforms import Compose, LoadImaged, EnsureChannelFirstd, ScaleIntensityd, ToTensord
from monai.data import CacheDataset, ThreadDataLoader
from monai.metrics import SSIMMetric, MAEMetric, PSNRMetric, RMSEMetric

# カスタムモジュール
from NNet.model_Nnet import Nnet
from NNet.Monai.TrainDataset_Nnet import Nnet_Dataset
from NNet.config import TRAINING_CONFIG, DATASET_CONFIG, MODEL_CONFIG, LOGGING_CONFIG, SCHEDULER_CONFIG, DEVICE_CONFIG
from NNet.utils import get_dataset_slice_counts, setup_device, save_model

# 警告を非表示に
warnings.filterwarnings("ignore")

2025-07-03 11:09:49.083077: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-03 11:09:49.083114: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-03 11:09:49.084174: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-03 11:09:49.090491: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-03 11:09:50.020209: W tensorflow/comp

## 2. 実験設定・再現性の確保

**目的**: 実験の再現性確保と結果管理のためのセットアップ
- **再現性**: シード値を固定してトレーニング結果を一貫させる
- **実験管理**: タイムスタンプ付きディレクトリで実験結果を整理
- **設定保存**: 実験パラメータを自動的にバックアップして再現可能性を確保

In [7]:
# 再現性のためのシード設定
set_determinism(seed=42)

# 実験ディレクトリの作成
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
experiment_dir = os.path.join("experiments", "Nnet", f"{timestamp}")
os.makedirs(experiment_dir, exist_ok=True)
print(f"実験ディレクトリ作成: {experiment_dir}")

# 設定の保存
def save_configs(experiment_dir):
    """実験ディレクトリに設定を保存"""
    config_dir = os.path.join(experiment_dir, "configs")
    os.makedirs(config_dir, exist_ok=True)

    configs = {
        "training_config": TRAINING_CONFIG,
        "dataset_config": DATASET_CONFIG,
        "model_config": MODEL_CONFIG,
        "logging_config": LOGGING_CONFIG,
        "scheduler_config": SCHEDULER_CONFIG,
        "device_config": DEVICE_CONFIG
    }
    
    for name, config in configs.items():
        with open(os.path.join(config_dir, f"{name}.txt"), "w") as f:
            f.write(f"{name}:\n")
            for key, value in config.items():
                f.write(f"{key}: {value}\n")

    print(f"設定ファイル保存先: {config_dir}")

save_configs(experiment_dir)

実験ディレクトリ作成: experiments/Nnet/20250703_103903
設定ファイル保存先: experiments/Nnet/20250703_103903/configs


## 3. デバイスの設定
使用するデバイス（GPU/CPU）を設定し、メトリクス計算器を初期化します。

In [7]:
# デバイスの設定
device = setup_device(DEVICE_CONFIG['use_cuda'], DEVICE_CONFIG['cuda_device'])

# メトリクス計算器の初期化
ssim_metric = SSIMMetric(spatial_dims=2, reduction="mean")
mae_metric = MAEMetric(reduction="mean")
psnr_metric = PSNRMetric(max_val=1.0, reduction="mean")
rmse_metric = RMSEMetric(reduction="mean")

print(f"使用デバイス: {device}")

Using device: cuda:0
GPU: NVIDIA GeForce RTX 3090
Memory: 25.4 GB
使用デバイス: cuda:0


## 4. ロギングの設定
TensorBoardとWeights & Biasesの設定を行います。

In [9]:
# TensorBoard設定
if LOGGING_CONFIG['use_tensorboard']:
    tb_log_dir = os.path.join(experiment_dir, "tensorboard")
    os.makedirs(tb_log_dir, exist_ok=True)
    tb_writer = SummaryWriter(tb_log_dir)
    print(f"TensorBoardログ保存先: {tb_log_dir}")
else:
    tb_writer = None

# Weights & Biases設定
use_wandb = False
if LOGGING_CONFIG['use_wandb']:
    run_name = os.path.basename(experiment_dir)
    try:
        wandb.init(
            project=LOGGING_CONFIG['wandb_project'],
            entity=LOGGING_CONFIG['wandb_entity'],
            config={
                **TRAINING_CONFIG,
                **DATASET_CONFIG,
                **MODEL_CONFIG,
                **SCHEDULER_CONFIG
            },
            name=run_name,
            dir=experiment_dir
        )
        use_wandb = True
        print("Weights & Biasesロギングを開始")
    except Exception as e:
        print(f"Weights & Biasesの初期化に失敗: {str(e)}")

TensorBoardログ保存先: experiments/Nnet/20250703_103903/tensorboard


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: guiju. Use `wandb login --relogin` to force relogin


Weights & Biasesロギングを開始


## 5. データセット準備・データローダー作成

**目的**: 医療CT画像データの効率的な読み込みと前処理
- **データ形式**: PNG画像ファイル（NPY形式にも対応可能）
- **前処理**: スケール正規化、チャンネル次元確保、テンソル変換
- **キャッシュ戦略**: メモリ使用量を考慮した部分キャッシュ（10%）
- **並列処理**: マルチワーカーによる高速データロード

**注意**: NPY形式を使用する場合は `TrainDataset_Nnet_NPY.py` と `LoadImaged(reader="numpyreader")` を使用

In [ ]:
# データ変換パイプラインの定義（PNG用、NPY用にも対応可能）
train_val_transforms = Compose([
    LoadImaged(keys=["img", "prior", "label"]),  # NPY使用時: reader="numpyreader"を追加
    EnsureChannelFirstd(keys=["img", "prior", "label"]),  # チャンネル次元を最初に配置
    ScaleIntensityd(keys=["img", "prior", "label"]),      # [0,1]範囲に正規化
    ToTensord(keys=["img", "prior", "label"])             # PyTorchテンソルに変換
])

# データルートパスの絶対パス化
DATASET_CONFIG['data_root'] = os.path.join(project_root, DATASET_CONFIG['data_root'])
print(f"データルート: {DATASET_CONFIG['data_root']}")

# トレーニングデータセットの準備
print("トレーニングデータセットを準備中...")
slice_counts_train = get_dataset_slice_counts(
    DATASET_CONFIG['data_root'],
    DATASET_CONFIG['train_dataset_indices'],
)
print(f"対象データセット: {DATASET_CONFIG['train_dataset_indices']}")
print(f"トレーニングスライス数: {slice_counts_train}")

train_dataset = Nnet_Dataset(
    DATASET_CONFIG['data_root'],  # 降質画像ルート
    DATASET_CONFIG['data_root'],  # 先験画像ルート
    DATASET_CONFIG['data_root'],  # GT画像ルート
    DATASET_CONFIG['train_dataset_indices'],
    slice_counts_train,
)

# バリデーションデータセットの準備
print("\nバリデーションデータセットを準備中...")
slice_counts_val = get_dataset_slice_counts(
    DATASET_CONFIG['data_root'],
    DATASET_CONFIG['val_dataset_indices'],
)
print(f"対象データセット: {DATASET_CONFIG['val_dataset_indices']}")
print(f"バリデーションスライス数: {slice_counts_val}")

val_dataset = Nnet_Dataset(
    DATASET_CONFIG['data_root'],
    DATASET_CONFIG['data_root'],
    DATASET_CONFIG['data_root'],
    DATASET_CONFIG['val_dataset_indices'],
    slice_counts_val,
)

# メモリ効率を考慮したキャッシュデータセットの作成
print("\nキャッシュデータセットを作成中...")
# 利用可能メモリに応じてcache_rateを調整可能（0.1 = 10%キャッシュ）
cache_rate = 0.1  
print(f"キャッシュ率: {cache_rate*100}% (メモリ使用量を抑制)")

train_dataset_cache = CacheDataset(
    data=train_dataset.samples,
    transform=train_val_transforms,
    cache_rate=cache_rate,
    num_workers=4,       # 並列処理ワーカー数
    progress=True        # プログレスバー表示
)

val_dataset_cache = CacheDataset(
    data=val_dataset.samples,
    transform=train_val_transforms,
    cache_rate=cache_rate,
    num_workers=2,
    progress=True
)

# 高効率データローダーの作成
print("\nデータローダーを作成中...")
train_loader = ThreadDataLoader(
    train_dataset_cache,
    batch_size=TRAINING_CONFIG['train_batch_size'],
    shuffle=True,                    # トレーニング時はシャッフル
    num_workers=TRAINING_CONFIG['num_workers'],
    drop_last=False,                 # 最後の不完全バッチも使用
    pin_memory=torch.cuda.is_available(),  # CUDA使用時はメモリ固定
    prefetch_factor=2,               # プリフェッチバッファサイズ
    persistent_workers=True          # ワーカープロセス永続化
)

val_loader = ThreadDataLoader(
    val_dataset_cache,
    batch_size=TRAINING_CONFIG['val_batch_size'],
    num_workers=TRAINING_CONFIG['num_workers'],
    pin_memory=torch.cuda.is_available(),
    prefetch_factor=2
)

print(f"\n📊 データセット統計:")
print(f"  トレーニングサンプル数: {len(train_dataset_cache):,}")
print(f"  バリデーションサンプル数: {len(val_dataset_cache):,}")
print(f"  トレーニングバッチ数: {len(train_loader)}")
print(f"  バリデーションバッチ数: {len(val_loader)}")
print(f"  バッチサイズ (train/val): {TRAINING_CONFIG['train_batch_size']}/{TRAINING_CONFIG['val_batch_size']}")

## 6. モデルの構築
N-netモデルを構築し、デバイスに移動します。

In [ ]:
# N-Netモデルの初期化とデバイス配置
print("N-Netモデルを初期化中...")
model = Nnet()
model = model.to(device)

# モデルパラメータ数の確認
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"📋 モデル情報:")
print(f"  総パラメータ数: {total_params:,}")
print(f"  訓練可能パラメータ数: {trainable_params:,}")
print(f"  モデルサイズ: {total_params * 4 / 1024**2:.1f} MB (float32)")

# モデルアーキテクチャの詳細表示
print("\n🏗️ モデルアーキテクチャ詳細:")
try:
    summary(model, [
        (MODEL_CONFIG['input_channels'], *DATASET_CONFIG['image_size']),  # 降質画像入力
        (MODEL_CONFIG['input_channels'], *DATASET_CONFIG['image_size'])   # 先験画像入力
    ])
except Exception as e:
    print(f"アーキテクチャ表示エラー: {str(e)}")
    print("モデル構造の簡略表示:")
    print(model)

# TensorBoardへのモデルグラフ追加（エラーハンドリング付き）
if tb_writer:
    try:
        print("\n📊 TensorBoardにモデルグラフを追加中...")
        # ダミー入力の作成（入力サイズに合わせて）
        dummy_input1 = torch.randn(
            1, MODEL_CONFIG['input_channels'], *DATASET_CONFIG['image_size']
        ).to(device)
        dummy_input2 = torch.randn(
            1, MODEL_CONFIG['input_channels'], *DATASET_CONFIG['image_size']
        ).to(device)
        
        # モデルグラフをTensorBoardに追加
        tb_writer.add_graph(model, (dummy_input1, dummy_input2))
        print("✅ モデルグラフをTensorBoardに正常に追加")
        
        # メモリ解放
        del dummy_input1, dummy_input2
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            
    except Exception as e:
        print(f"⚠️ TensorBoardグラフ追加に失敗: {str(e)}")

print("\n✅ モデル構築完了")

## 7. オプティマイザーと損失関数の設定
トレーニングに必要なオプティマイザー、スケジューラー、損失関数を設定します。

In [ ]:
# トレーニング設定の計算と表示
batches_per_epoch = len(train_loader)
total_batches = TRAINING_CONFIG['epochs'] * batches_per_epoch
print(f"📈 トレーニング設定:")
print(f"  エポック数: {TRAINING_CONFIG['epochs']}")
print(f"  バッチ数/エポック: {batches_per_epoch}")
print(f"  総バッチ数: {total_batches:,}")

# Adamオプティマイザーの設定
print(f"\n⚙️ オプティマイザー設定:")
print(f"  種類: Adam")
print(f"  初期学習率: {SCHEDULER_CONFIG['max_lr']}")
optimizer = optim.Adam(
    model.parameters(), 
    lr=SCHEDULER_CONFIG['max_lr'],
    betas=(0.9, 0.999),           # Adam momentum parameters
    eps=1e-8,                     # numerical stability
    weight_decay=0                # L2正則化（必要に応じて調整）
)

# コサイン学習率スケジューラーの設定
print(f"\n📉 学習率スケジューラー設定:")
print(f"  種類: CosineAnnealingLR")
print(f"  最大学習率: {SCHEDULER_CONFIG['max_lr']}")
print(f"  最小学習率: {SCHEDULER_CONFIG['min_lr']}")
print(f"  総ステップ数: {total_batches:,}")

scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=total_batches,          # 全トレーニングステップ数
    eta_min=SCHEDULER_CONFIG['min_lr'],  # 最小学習率
    last_epoch=-1                 # 初期値
)

# 医療画像専用損失関数の設定
print(f"\n🎯 損失関数設定:")
print(f"  複合損失関数（4つの損失の重み付き組み合わせ）")

# 各損失関数の初期化
mse_loss = MSELoss().to(device)
l1_loss = L1Loss().to(device)
ssim_loss = SSIMLoss(spatial_dims=2).to(device)
perceptual_loss = PerceptualLoss(spatial_dims=2, network_type="alex").to(device)

# 損失重みの設定（config.pyから取得）
loss_weights = {
    'mse': TRAINING_CONFIG['weight_mse'],       # 画像再構築の基本損失
    'l1': TRAINING_CONFIG['weight_l1'],         # エッジ保持のためのL1損失
    'ssim': TRAINING_CONFIG['weight_ssim'],     # 構造類似性損失
    'perceptual': TRAINING_CONFIG['weight_percep']  # 知覚的損失
}

print(f"  損失重み: MSE={loss_weights['mse']}, L1={loss_weights['l1']}, SSIM={loss_weights['ssim']}, Perceptual={loss_weights['perceptual']}")

# 医療画像特化の複合損失関数
def combined_loss(pred, target):
    """
    4つの損失関数を組み合わせた医療画像専用損失
    
    Args:
        pred: モデル予測結果 [B, C, H, W]
        target: 正解画像 [B, C, H, W]
    
    Returns:
        combined_loss: 重み付き損失の合計
    """
    try:
        # 各損失の計算
        mse = mse_loss(pred, target)          # Mean Squared Error
        l1 = l1_loss(pred, target)            # L1 Loss (MAE)
        ssim = ssim_loss(pred, target)        # Structural Similarity
        percep = perceptual_loss(pred, target) # Perceptual Loss
        
        # 重み付きで組み合わせ
        total_loss = (loss_weights['mse'] * mse + 
                     loss_weights['l1'] * l1 + 
                     loss_weights['perceptual'] * percep + 
                     loss_weights['ssim'] * ssim)
        
        return total_loss
        
    except Exception as e:
        print(f"損失計算エラー: {str(e)}")
        # フォールバック: MSE損失のみ
        return mse_loss(pred, target)

print(f"\n✅ 最適化設定完了")

## 8. メトリクス計算関数
バリデーション用のメトリクス計算関数を定義します。

In [ ]:
def calculate_metrics(pred, target):
    """
    医療画像評価指標の効率的な計算
    
    医療画像の品質評価に重要な4つの指標を計算します：
    - RMSE: 根平均二乗誤差（画素レベルの再構築精度）
    - MAE: 平均絶対誤差（平均的な画素差）
    - PSNR: ピーク信号対雑音比（画像品質の客観評価）
    - SSIM: 構造類似性指数（人間の視覚的認識に近い評価）
    
    Args:
        pred: モデル予測結果 [B, C, H, W]
        target: 正解画像 [B, C, H, W]
    
    Returns:
        dict: 各評価指標の値を含む辞書
    """
    try:
        # データ型の統一（計算精度向上のため）
        pred = pred.float()
        target = target.float()
        
        # 各メトリクスのリセット（MONAI特有の要件）
        ssim_metric.reset()
        mae_metric.reset()
        psnr_metric.reset()
        rmse_metric.reset()
        
        # バッチ単位でのメトリクス計算
        # SSIM: 構造類似性（-1〜1、1が最良）
        ssim_metric(pred, target)
        ssim_val = ssim_metric.aggregate().item()
        
        # MAE: 平均絶対誤差（0が最良）
        mae_metric(pred, target)
        mae_val = mae_metric.aggregate().item()
        
        # PSNR: ピーク信号対雑音比（高いほど良い、通常20-50dB）
        psnr_metric(pred, target)
        psnr_val = psnr_metric.aggregate().item()
        
        # RMSE: 根平均二乗誤差（0が最良）
        rmse_metric(pred, target)
        rmse_val = rmse_metric.aggregate().item()
        
        return {
            "rmse": rmse_val,
            "mae": mae_val,
            "psnr": psnr_val,
            "ssim": ssim_val
        }
        
    except Exception as e:
        print(f"⚠️ メトリクス計算エラー: {str(e)}")
        # エラー時のフォールバック値
        return {
            "rmse": float('inf'),
            "mae": float('inf'),
            "psnr": 0.0,
            "ssim": 0.0
        }

def format_metrics_string(metrics_dict, precision=6):
    """
    メトリクス辞書を見やすい文字列に変換
    
    Args:
        metrics_dict: calculate_metricsの返り値
        precision: 小数点以下の桁数
    
    Returns:
        str: フォーマット済み文字列
    """
    return (f"RMSE: {metrics_dict['rmse']:.{precision}f}, "
            f"MAE: {metrics_dict['mae']:.{precision}f}, "
            f"PSNR: {metrics_dict['psnr']:.{precision}f}dB, "
            f"SSIM: {metrics_dict['ssim']:.{precision}f}")

print("✅ メトリクス計算関数を定義完了")

## 9. メイントレーニングループ実行

**目的**: N-Netモデルの効率的なトレーニング実行
- **トレーニング**: フォワード・バックワードパス、勾配最適化
- **バリデーション**: 過学習監視とベストモデル保存
- **ログ記録**: TensorBoard・W&Bでの詳細なメトリクス追跡
- **メモリ管理**: OOM対応とGPUメモリ効率化
- **進捗表示**: リアルタイムでの学習状況可視化

**重要パラメータ**:
- 勾配クリッピング: `max_norm=1.0` （勾配爆発防止）
- プログレス表示: 10バッチごと（計算効率とログ詳細度のバランス）
- ベストモデル保存: バリデーション損失最小時に自動保存

In [16]:
# ベストモデルの追跡変数
best_val_loss = float('inf')
best_val_metrics = None
best_epoch = 0

print("トレーニングを開始します...")

for epoch in range(TRAINING_CONFIG['epochs']):
    print(f"\nエポック {epoch + 1}/{TRAINING_CONFIG['epochs']}")
    print("-" * 50)
    
    # --- トレーニングフェーズ ---
    model.train()
    print(f"[エポック {epoch + 1}] トレーニング中")
    
    for i, batch in enumerate(train_loader):
        # データをデバイスに移動
        images = batch["img"].to(device)
        prior = batch["prior"].to(device)
        labels = batch["label"].to(device)
        
        # 勾配のリセット
        optimizer.zero_grad()
        
        try:
            # フォワードパス
            prediction = model(images, prior)
            loss = combined_loss(prediction, labels)
            
            # バックワードパスとパラメータ更新
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()
            
            # メトリクスの計算
            batch_metrics = calculate_metrics(prediction, labels)
            
            # 進捗表示
            if (i + 1) % 10 == 0:  # 10バッチごとに表示
                print(f"[エポック {epoch+1} バッチ {i+1}] 損失: {loss.item():.6f}, "
                      f"RMSE: {batch_metrics['rmse']:.6f}, MAE: {batch_metrics['mae']:.6f}")
            
            # TensorBoardロギング
            if tb_writer:
                global_step = epoch * len(train_loader) + i
                tb_writer.add_scalar('Train/Loss', loss.item(), global_step)
                tb_writer.add_scalar('Train/RMSE', batch_metrics['rmse'], global_step)
                tb_writer.add_scalar('Train/MAE', batch_metrics['mae'], global_step)
                tb_writer.add_scalar('Train/PSNR', batch_metrics['psnr'], global_step)
                tb_writer.add_scalar('Train/SSIM', batch_metrics['ssim'], global_step)
            
            # Weights & Biasesロギング
            if use_wandb:
                wandb.log({
                    'train_loss': loss.item(),
                    'train_rmse': batch_metrics['rmse'],
                    'train_mae': batch_metrics['mae'],
                    'train_psnr': batch_metrics['psnr'],
                    'train_ssim': batch_metrics['ssim'],
                    'epoch': epoch + 1
                })
                
        except RuntimeError as e:
            if "out of memory" in str(e):
                print("警告: メモリ不足 - キャッシュをクリアして継続")
                optimizer.zero_grad()
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                continue
            else:
                raise e
    
    # --- バリデーションフェーズ ---
    model.eval()
    print(f"[エポック {epoch + 1}] バリデーション中")
    
    running_loss = 0.0
    running_metrics = {'rmse': 0.0, 'mae': 0.0, 'psnr': 0.0, 'ssim': 0.0}
    num_batches = 0
    
    with torch.no_grad():
        for batch in val_loader:
            images = batch["img"].to(device)
            prior = batch["prior"].to(device)
            labels = batch["label"].to(device)
            
            # 予測と損失計算
            prediction = model(images, prior)
            loss = combined_loss(prediction, labels)
            
            # 損失とメトリクスの蓄積
            running_loss += loss.item()
            batch_metrics = calculate_metrics(prediction, labels)
            
            for key in running_metrics:
                running_metrics[key] += batch_metrics[key]
            
            num_batches += 1
    
    # 平均値の計算
    avg_loss = running_loss / num_batches
    avg_metrics = {k: v / num_batches for k, v in running_metrics.items()}
    
    # 結果表示
    print(f"[エポック {epoch + 1}] 検証損失: {avg_loss:.6f}, "
          f"RMSE: {avg_metrics['rmse']:.6f}, MAE: {avg_metrics['mae']:.6f}, "
          f"PSNR: {avg_metrics['psnr']:.6f}, SSIM: {avg_metrics['ssim']:.6f}")
    
    # ベストモデルのチェックと保存
    if avg_loss < best_val_loss:
        best_val_loss = avg_loss
        best_val_metrics = avg_metrics
        best_epoch = epoch + 1
        
        model_save_dir = os.path.join(experiment_dir, TRAINING_CONFIG['model_save_dir'])
        os.makedirs(model_save_dir, exist_ok=True)
        
        save_model(
            model,
            epoch + 1,
            avg_loss,
            model_save_dir,
            'nnet_medical_ct_best'
        )
        print(f"ベストモデルをエポック {epoch+1} で保存")
    
    # ベスト結果の表示
    print(f"[ベスト検証結果] エポック: {best_epoch}, 損失: {best_val_loss:.6f}, "
          f"RMSE: {best_val_metrics['rmse']:.6f}, MAE: {best_val_metrics['mae']:.6f}")
    
    # TensorBoardロギング
    if tb_writer:
        tb_writer.add_scalar('Val/Loss', avg_loss, epoch)
        tb_writer.add_scalar('Val/RMSE', avg_metrics['rmse'], epoch)
        tb_writer.add_scalar('Val/MAE', avg_metrics['mae'], epoch)
        tb_writer.add_scalar('Val/PSNR', avg_metrics['psnr'], epoch)
        tb_writer.add_scalar('Val/SSIM', avg_metrics['ssim'], epoch)
        tb_writer.add_scalar('Learning_Rate', optimizer.param_groups[0]['lr'], epoch)
        
        # ベストモデルの画像可視化
        if epoch > 5 and avg_loss == best_val_loss:
            img_cpu = images[0].cpu().float()
            prior_cpu = prior[0].cpu().float()
            label_cpu = labels[0].cpu().float()
            pred_cpu = prediction[0].cpu().float()
            
            grid = torchvision.utils.make_grid(
                [img_cpu, prior_cpu, label_cpu, pred_cpu],
                nrow=4,
                normalize=True
            )
            tb_writer.add_image(f"Validation/Artifact_Removal", grid, epoch)
    
    # Weights & Biasesロギング
    if use_wandb:
        wandb.log({
            'val_loss': avg_loss,
            'val_rmse': avg_metrics['rmse'],
            'val_mae': avg_metrics['mae'],
            'val_psnr': avg_metrics['psnr'],
            'val_ssim': avg_metrics['ssim'],
            'learning_rate': optimizer.param_groups[0]['lr'],
            'epoch': epoch + 1
        })

print("トレーニングが正常に完了しました!")

トレーニングを開始します...

エポック 1/40
--------------------------------------------------
[エポック 1] トレーニング中


KeyboardInterrupt: 

## 10. トレーニング完了・リソース解放

**目的**: トレーニング終了後の適切なリソース管理
- **ロギング終了**: TensorBoard・W&Bセッションの正常終了
- **メモリ解放**: GPU・CPUメモリの効率的な解放
- **結果保存**: 最終モデルと実験ログの確認

**重要**: 実験結果は `experiment_dir` に自動保存されます
- モデル重み: `trained_model/` ディレクトリ
- TensorBoard: `tensorboard/` ディレクトリ  
- 設定ファイル: `configs/` ディレクトリ
- W&Bログ: オンラインダッシュボードで確認可能

In [ ]:
print("🧹 リソースクリーンアップを実行中...")

# TensorBoardライターの安全な終了
if tb_writer:
    try:
        tb_writer.close()
        print("✅ TensorBoardライターを正常に終了")
    except Exception as e:
        print(f"⚠️ TensorBoard終了エラー: {str(e)}")
else:
    print("ℹ️ TensorBoardは使用されていません")

# Weights & Biasesセッションの終了
if use_wandb:
    try:
        wandb.finish()
        print("✅ Weights & Biasesセッションを正常に終了")
    except Exception as e:
        print(f"⚠️ W&B終了エラー: {str(e)}")
else:
    print("ℹ️ Weights & Biasesは使用されていません")

# GPUメモリの解放
if torch.cuda.is_available():
    try:
        # キャッシュされたメモリを解放
        torch.cuda.empty_cache()
        
        # GPU使用量の表示
        gpu_memory_used = torch.cuda.memory_allocated(device) / 1024**3  # GB
        gpu_memory_cached = torch.cuda.memory_reserved(device) / 1024**3  # GB
        
        print(f"✅ GPUメモリを解放完了")
        print(f"  現在の使用量: {gpu_memory_used:.2f} GB")
        print(f"  キャッシュ済み: {gpu_memory_cached:.2f} GB")
        
    except Exception as e:
        print(f"⚠️ GPUメモリ解放エラー: {str(e)}")
else:
    print("ℹ️ GPUは使用されていません")

# 実験結果の確認とサマリー表示
print(f"\n📁 実験結果ディレクトリ: {experiment_dir}")
print(f"📋 実験サマリー:")

try:
    # 保存されたファイルの確認
    if os.path.exists(experiment_dir):
        subdirs = ['tensorboard', 'configs', 'trained_model']
        for subdir in subdirs:
            subdir_path = os.path.join(experiment_dir, subdir)
            if os.path.exists(subdir_path):
                file_count = len([f for f in os.listdir(subdir_path) 
                                if os.path.isfile(os.path.join(subdir_path, f))])
                print(f"  {subdir}/: {file_count} ファイル")
            else:
                print(f"  {subdir}/: 作成されませんでした")
                
        print(f"\n🎉 トレーニング完了！")
        print(f"📊 TensorBoard表示: tensorboard --logdir {os.path.join(experiment_dir, 'tensorboard')}")
        
        if use_wandb:
            print(f"🌐 W&Bダッシュボード: https://wandb.ai/{LOGGING_CONFIG.get('wandb_entity', 'your-entity')}/{LOGGING_CONFIG.get('wandb_project', 'your-project')}")
            
except Exception as e:
    print(f"⚠️ 実験ディレクトリ確認エラー: {str(e)}")

print(f"\n✨ 全ての処理が正常に完了しました！")